# Assignment 3

Load the CIFAR-10 dataset from Keras. Normalize the data and split it into training, validaiton and test sets.

In [20]:
import keras
# Load the CIFAR-10 dataset
(inputs, targets), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Convert targets to categorical format
targets = keras.utils.to_categorical(targets, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

# Normalize the data
inputs = inputs.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Split the data into training, validation, and test sets
x_train = inputs[:40000]
y_train = targets[:40000]

x_val = inputs[40000:50000]
y_val = targets[40000:50000]

print(f"Training data shape: {x_train.shape}, {y_train.shape}")
print(f"Validation data shape: {x_val.shape}, {y_val.shape}")
print(f"Test data shape: {x_test.shape}, {y_test.shape}")

Training data shape: (40000, 32, 32, 3), (40000, 10)
Validation data shape: (10000, 32, 32, 3), (10000, 10)
Test data shape: (10000, 32, 32, 3), (10000, 10)
